# Aula 18 – Análise de Componentes Principais


## 1. Introdução

Uma imagem pode ser entendida simplesmente como um conjunto de dados espacialmente distribuídos (as intensidades dos pixels são os dados). Quando fazemos visão computacional, queremos analisar esses dados e identificar o que eles significam. A maior dificuldade disso está no fato de que são muitos, tornando inviável identificar padrões diretamente nessa distribuição. Todas as técnicas discutidas nas aulas anteriores tinham um objetivo principal: limpar a imagem para que pudessem ser tratados apenas os dados (pixels) que realmente interessam em um determinado processo de identificação. 

Entretanto, mesmo após todo pré-processamento, ainda assim a quantidade de dados a ser analisada pode ser muito grande, ou de “alta dimensionalidade”. Pode ser desejável, então, diminuir ainda mais a quantidade de dados, ou seja, fazer uma redução de dimensionalidade, sem, entretanto, descaracterizá-los, ou seja, que o conjunto menor de dados reduzido ainda traga a informação necessária para identificar o padrão da imagem.

A técnica de Análise de Componentes Principais (PCA – Principal Component Analysis) é uma técnica estatística que realiza a tarefa desejada de redução de dimensionalidade. Matematicamente, é uma transformação linear que transforma os dados para um novo sistema de coordenadas, onde a projeção dos dados de acordo com a maior variância se dá na primeira coordenada (primeira componente), a segunda maior variância na segunda coordenada (segunda componente) e assim por diante. 

Observe que um conjunto de dados com alta variância é mais importante para a caracterização de uma imagem do que baixa variância (ou parecidos), que trará pouca informação sobre o padrão contido nos dados. Assim, podemos analisar a imagem considerando os pixels como dados no novo espaço gerado pelo PCA. Como os eixos estão ordenados por variância, podemos escolher as “primeiras” coordenadas do PCA apenas para analisar (só a primeira, as duas primeiras, as três primeiras…), pois elas caracterizarão melhor a imagem (a quantidade de componentes a utilizar  depende da aplicação e pode ser um parâmetro dado).

Cada eixo é definido por um vetor. A redução de dimensionalidade acontece porque você substitui todos os dados apenas pelas informações das “direções” de maior variância deles. Ou seja, os vetores que definem os eixos do PCA são os seus dados agora. Tais vetores são especificados no que chamamos “espaço de características” da imagem.

A técnica de PCA apenas diminui a dimensionalidade dos dados, mas isso por si só não dá a identificação de uma imagem. Entretanto, com um número menor de dados a analisar, podemos aplicar agora algoritmos para identificação que sejam mais eficientes. A partir da próxima aula finalmente passaremos, de fato, a aprender como fazer o computador “ver” uma imagem, ou seja, identificá-la. Ainda, embora PCA em si não sirva para identificar objetos em uma imagem, é possível utilizá-lo por si só para identificar orientações de objetos, o que é uma aplicação também importante em visão computacional.

## 2. Leitura Complementar

Livro 1 – Seções 8.3 e 8.4

Livro 3 – Seção 11.4

## 3. Exercícios

Realize o tutorial de PCA em OpenCV: https://docs.opencv.org/3.4/d1/dee/tutorial_introduction_to_pca.html

In [5]:
from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np
import argparse
from math import atan2, cos, sin, sqrt, pi

In [6]:
def drawAxis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    ## [visualization1]
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))

    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)

    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)

    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv.LINE_AA)
    ## [visualization1]

def getOrientation(pts, img):
    ## [pca]
    # Construct a buffer used by the pca analysis
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]

    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv.PCACompute2(data_pts, mean)

    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    ## [pca]

    ## [visualization]
    # Draw the principal components
    cv.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (0, 255, 0), 1)
    drawAxis(img, cntr, p2, (255, 255, 0), 5)

    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    ## [visualization]

    return angle

In [7]:
## [pre-process]
# Load image
parser = argparse.ArgumentParser(description='Code for Introduction to Principal Component Analysis (PCA) tutorial.\
                                              This program demonstrates how to use OpenCV PCA to extract the orientation of an object.')
parser.add_argument('--input', help='Path to input image.', default='moedasw.jpg')
args = parser.parse_args()

src = cv.imread(cv.samples.findFile(args.input))
# Check if image is loaded successfully
if src is None:
    print('Could not open or find the image: ', args.input)
    exit(0)

cv.imshow('src', src)

# Convert image to grayscale
gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)

# Convert image to binary
_, bw = cv.threshold(gray, 50, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
## [pre-process]

## [contours]
# Find all the contours in the thresholded image
_, contours, _ = cv.findContours(bw, cv.RETR_LIST, cv.CHAIN_APPROX_NONE)

for i, c in enumerate(contours):
    # Calculate the area of each contour
    area = cv.contourArea(c)
    # Ignore contours that are too small or too large
    if area < 1e2 or 1e5 < area:
        continue

    # Draw each contour only for visualisation purposes
    cv.drawContours(src, contours, i, (0, 0, 255), 2)
    # Find the orientation of each shape
    getOrientation(c, src)
## [contours]

cv.imshow('output', src)
cv.waitKey()

usage: ipykernel_launcher.py [-h] [--input INPUT]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/vando/.local/share/jupyter/runtime/kernel-eec9d0fa-84ae-45a5-bcee-5bdac287b481.json


SystemExit: 2